# Setup

In [ ]:
# Global Settings
SEED = 1
VERBOSE_LEVEL = 1
DIM = 768
DIM_RESIZE = 512
EPOCHS = 100
BATCH_SIZE = 32
NUM_CLASSES = 1

LR_MAX = 1e-4
LR_MIN = 1e-6
LR_START = 1e-5
FIND_LR = False
SAVE_OUTPUT = True
USE_TENSORBOARD = False

IMAGE_SIZE = [DIM, DIM]
IMAGE_RESIZE = [DIM_RESIZE, DIM_RESIZE]

In [ ]:
%%capture
!pip install tensorflow_addons
!pip install efficientnet

In [ ]:
import os
import re
import gc
import sys
import glob
import math
import json
import shutil
import random
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from pathlib import Path
from functools import partial
import matplotlib.pyplot as plt
import efficientnet.keras as efn
from IPython.display import Image
from datetime import datetime, date
from pandas.core.common import flatten
from IPython.display import clear_output
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model


In [ ]:
# cleanup
for f in glob.glob("/content/*.png"):
    os.remove(f)

for f in glob.glob("/content/*.hdf5"):
    os.remove(f)

for f in glob.glob("/content/*.txt"):
    os.remove(f)

for f in glob.glob("/content/*.h5"):
    os.remove(f)

for f in glob.glob("/content/*.json"):
    os.remove(f)

!rm -rf logs

In [ ]:
%%capture
!rm -rf /content/melanoma-classification
!rm -rf /content/sample_data/

# if we do not have the data
if not os.path.exists("/content/melanoma-classification/README.md"):
    # install kaggle
    !pip install kaggle

    # get kaggle auth file
    path_to_auth_file = "kaggle.json"
    if not os.path.exists(path_to_auth_file):
        path_to_auth_file = '/content/drive/MyDrive/Colab Notebooks/_auth/kaggle.json'

    # read the kaggle.json file
    with open(path_to_auth_file) as json_file:
        data = json.load(json_file)
        os.environ['KAGGLE_USERNAME'] = data["username"]
        os.environ['KAGGLE_KEY'] = data["key"]

    # remove sample data so we have more space
    !rm -rf /content/sample_data

    # download kaggle dataset
    !kaggle datasets download -d saschamet/melanoma-1-12
    !unzip /content/melanoma-1-12.zip
    !rm /content/melanoma-1-12.zip

    ## pull repo
    !git clone https://github.com/SaschaMet/melanoma-classification.git

    ## update repo
    !cd melanoma-classification && git fetch && git pull

    ## switch branch and execute function
    !cd /content/melanoma-classification && git checkout "master" && git fetch && git reset --hard origin/master

In [ ]:
# make sure the repo ist added to the sys path
sys.path.insert(0,'/content/melanoma-classification')
sys.path.insert(1,'/content/melanoma-classification/src')

In [ ]:
# Tensorflow execution optimizations
# Source: https://www.tensorflow.org/guide/mixed_precision & https://www.tensorflow.org/xla
tpu = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
except:
    pass

if tpu:
    print("Try connecting to a tpu")
    try:
        print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
        # tf.config.experimental_connect_to_cluster(tpu)
        # tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        REPLICAS = strategy.num_replicas_in_sync
        print("REPLICAS:", REPLICAS)
    except ValueError as error:
        raise BaseException('An exception occurred: {}'.format(error))
    except BaseException as error:
        raise BaseException('An exception occurred: {}'.format(error))
else:
    num_gpus = len(
        tf.config.experimental.list_physical_devices('GPU')
    )
    print("Using default strategy for CPU and single GPU")
    print("Num GPUs Available: ", num_gpus)
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')
    strategy = tf.distribute.get_strategy()
    REPLICAS = strategy.num_replicas_in_sync
    print("REPLICAS:", REPLICAS)

In [ ]:
if tpu:
    BATCH_SIZE = 128  # increase the batch size if we have a tpu
    USE_TENSORBOARD = False # Tensorboard does not work with tpu

# Set needed env variables based on the global variables
os.environ["DIM"] = str(DIM)
os.environ["BATCH_SIZE"] = str(BATCH_SIZE)

# seed everything
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# get the current timestamp. This timestamp is used to save the model data with a unique name
now = datetime.now()
today = date.today()
current_time = now.strftime("%H:%M:%S")
timestamp = str(today) + "_" + str(current_time)

# environment settings
print("Tensorflow version " + tf.__version__)
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
%load_ext autoreload
%autoreload 2

# Custom imports
from model.evaluation import evaluate_model
from model.model_callbacks import get_model_callbacks
from model.LearningRateFinder import LearningRateFinder
from data.verify_tf_records import display_batch_of_images
from model.clr_callback import cyclic_learning_rate, get_lr_callback, plot_clr

In [ ]:
## prevent colab shutdown 
%%javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}

var connection = setInterval(ConnectButton, 60000);

function myStopFunction() {
    console.log("Remove interal"); 
    clearInterval(connection);
}

# Data Loading

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
MALIGNANT_IMAGES = tf.io.gfile.glob('gs://kds-dd07414d960d14a2a8849e3ab696a4cb3299162b439e521ac886f393/*.tfrec')
print("MALIGNANT TF Records", len(MALIGNANT_IMAGES))

# This files include all malginant images from 2020, 2019, 2018 and 2017 competition as well as data directly from ISIC
# Source: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/169139
MALIGNANT_IMAGES_V2 = tf.io.gfile.glob('gs://kds-77aee4ba270735bf8687a77209581d50b8710ff6486f134f19b29759/*train*.tfrec')[15:]
print("MALIGNANT TF Records V2", len(MALIGNANT_IMAGES_V2))

BENIGN_IMAGES = tf.io.gfile.glob('gs://kds-d499b3e548bc098dfea46bcac127fb30d0e465c2713c8457ff05deff/*.tfrec')
print("BENIGN TF Records", len(BENIGN_IMAGES))

BENIGN_IMAGES_TRAIN = BENIGN_IMAGES[0:160] # use only the first 160 batches for training

TEST_FILENAMES = tf.io.gfile.glob("gs://kds-3e36e1551f5588596f9fcb50129d35830a155849a24ad4825763d528/*.tfrec")
print("TEST TF Records", len(TEST_FILENAMES))

# Create the Training and Validation Dataset
print(" ")

## Training 
TRAINING_FILENAMES = [MALIGNANT_IMAGES[1], MALIGNANT_IMAGES[0]]
TRAINING_FILENAMES = TRAINING_FILENAMES + random.sample(MALIGNANT_IMAGES_V2, 20)
TRAINING_FILENAMES = TRAINING_FILENAMES + random.sample(BENIGN_IMAGES_TRAIN, len(TRAINING_FILENAMES))
np.random.shuffle(TRAINING_FILENAMES)

## Validation 
VALIDATION_FILENAMES = [
    MALIGNANT_IMAGES[2],
    BENIGN_IMAGES[162]
]

TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)

print("TRAINING FILES", len(TRAINING_FILENAMES))
print("VALODATION FILES", len(VALIDATION_FILENAMES))
print(" ")

print("TRAINING_IMAGES", TRAINING_IMAGES)
print("VALIDATION_IMAGES", VALIDATION_IMAGES)

In [ ]:
AUGMENTATION_CONFIG = dict(
    # DATA AUGMENTATION
    rot = 180.0,
    shr = 1.5,
    hzoom = 6.0,
    wzoom = 6.0,
    hshift = 6.0,
    wshift = 6.0,
    
    # COARSE DROPOUT
    DROP_FREQ = 0, # Determines proportion of train images to apply coarse dropout to / Between 0 and 1.
    DROP_CT = 0, # How many squares to remove from train images when applying dropout.
    DROP_SIZE = 0, # The size of square side equals IMG_SIZE * DROP_SIZE / Between 0 and 1.  
)

def color(x):
    """Color augmentation
    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_saturation(x, 0.7, 1.7, seed=SEED)
    x = tf.image.random_brightness(x, 0.3, seed=SEED)
    x = tf.image.random_contrast(x, 0.5, 1.5, seed=SEED)
    return x

def flip(x):
    """Flip augmentation
    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x, seed=SEED)
    x = tf.image.random_flip_up_down(x, seed=SEED)
    return x

def rotate(x):
    """Rotation augmentation
    Args:
        x: Image

    Returns:
        Augmented image
    """
    return tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))


def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))
    

def transform(image, cfg=AUGMENTATION_CONFIG):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = DIM_RESIZE
    XDIM = DIM%2 #fix for size 331
    
    rot = cfg['rot'] * tf.random.normal([1], dtype='float32')
    shr = cfg['shr'] * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / cfg['hzoom']
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / cfg['wzoom']
    h_shift = cfg['hshift'] * tf.random.normal([1], dtype='float32') 
    w_shift = cfg['wshift'] * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift)

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM_RESIZE, DIM_RESIZE,3])

In [ ]:
def augment_image(image, augment=True):  
    augmentations = [transform, color, rotate, flip] 
    if augment:
        # Data augmentation
        for f in augmentations:
            if random.randint(1, 10) >= 5:
                image = f(image)         
    return image

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    return image

In [ ]:
resizing_layer = tf.keras.layers.experimental.preprocessing.Resizing(DIM_RESIZE, DIM_RESIZE)

In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.cache() # cache ds for performance gains
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(lambda x, y: (resizing_layer(x), y), num_parallel_calls=AUTOTUNE) # resize the images to the same height and width

    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

In [ ]:
def get_training_dataset(augment):
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    if augment:
        dataset = dataset.map(lambda x, y: (augment_image(x, augment=augment), y), num_parallel_calls=AUTOTUNE)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(TRAINING_IMAGES * 2, reshuffle_each_iteration=True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_validation_dataset(ordered=False, repeat=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    if repeat:
        dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

# Data Validation

In [ ]:
example_dataset = get_training_dataset(augment=False)
example_dataset = example_dataset.unbatch().batch(15)

In [ ]:
example_batch = iter(example_dataset) 
image_batch, label_batch = next(example_batch)
# because Xception needs a special kind op preprocessing, which plt cannot display correctly,
# we do some image processing for plt here
images = [(x / 255) for x in image_batch]
labels = [l.numpy() for l in label_batch]
display_batch_of_images((images, labels), unbatched=True)

In [ ]:
augmented_images = [augment_image(x, augment=True) for x in images]
augmented_images = [np.clip(x, 0, 1) for x in augmented_images]
labels = [l.numpy() for l in label_batch]
display_batch_of_images((augmented_images, labels), unbatched=True)

In [ ]:
# images are in float32 format with values between 0 and 255
for i in range(10):
    image = image_batch[i]
    print("min:", np.min(image), " -  max:", np.max(image))

print(image.dtype)

In [ ]:
# Iterate over n batches to get the class distribution
benign_cases = 0
malignant_cases = 0

for i in range(0, 50):
    x,y = next(example_batch)
    for label in y.numpy():
        if int(label) == 0:
            benign_cases = benign_cases + 1
        else:
            malignant_cases = malignant_cases + 1


initial_bias = np.log([malignant_cases/benign_cases])

weight_for_0 = (1 / benign_cases)*(TRAINING_IMAGES)/2.0 
weight_for_1 = (1 / malignant_cases)*(TRAINING_IMAGES)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print("benign_cases", benign_cases)
print("malignant_cases", malignant_cases)
print("ratio", round(malignant_cases / benign_cases, 2))
print("initial_bias", initial_bias)

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

# Model Creation

In [ ]:
training_dataset = get_training_dataset(augment=True)
validation_dataset = get_validation_dataset(repeat=True)

steps_per_epoch = 30
validation_steps_per_epoch = 5

print("Epochs", EPOCHS)
print("BATCH SIZE", BATCH_SIZE)

print("Train images", count_data_items(TRAINING_FILENAMES))
print("Validation images", count_data_items(VALIDATION_FILENAMES))

print("steps_per_epoch", steps_per_epoch)
print("validation_steps_per_epoch", validation_steps_per_epoch)

In [ ]:
def get_model_parameters(lr):
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    loss = 'binary_crossentropy'
    metrics = [
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.AUC(name='auc'),
    ]

    return loss, metrics, optimizer

In [ ]:
def compile_model(model):
    loss, metrics, optimizer = get_model_parameters(LR_START)
    if tpu:
        model.compile(
            loss=loss,
            metrics=metrics,
            optimizer=optimizer,
            # Reduce python overhead, and maximize the performance of your TPU
            # Anything between 2 and `steps_per_epoch` could help here.
            steps_per_execution=int(steps_per_epoch / 10),
        )
    else:
        model.compile(
            loss=loss,
            metrics=metrics,
            optimizer=optimizer,
        )

    return model


def reset_model():
    model = tf.keras.models.load_model("stage_0.h5")
    model.load_weights("stage_0.hdf5")
    return model

In [ ]:
# Clear the session - this helps when we are creating multiple models
K.clear_session()

# Creating the model in the strategy scope places the model on the TPU
with strategy.scope():

    i = tf.keras.layers.Input([DIM_RESIZE, DIM_RESIZE, 3], dtype = tf.float32)
    x = tf.keras.applications.xception.preprocess_input(i)
    base_model = tf.keras.applications.Xception(
        input_shape=(DIM_RESIZE, DIM_RESIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    x = base_model(x)
    pretrained_model = tf.keras.Model(inputs=[i], outputs=[x], name="xception")

    base_model.trainable = False

    if initial_bias is not None:
        output_bias = tf.keras.initializers.Constant(initial_bias)

    model = tf.keras.models.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid', bias_initializer=output_bias)
    ])

    model = compile_model(model)

In [ ]:
#plot_model(base_model, show_shapes=True)

In [ ]:
model.summary()

# Initial Model Training

In [ ]:
history = model.fit(
    training_dataset,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    validation_steps=validation_steps_per_epoch,
    class_weight=class_weight,
    verbose=VERBOSE_LEVEL
)

In [ ]:
# save the initially trained model, so we can restore it later
model.save_weights("stage_0.hdf5")
model.save("stage_0.h5")

# Learning Rate Finder

In [ ]:
if FIND_LR:
    K.clear_session()
    with strategy.scope():
        print("Current LR Min", LR_MIN)
        print("Current LR Max", LR_MAX)

        model = reset_model()
        model.trainable = True
        model = compile_model(model)

        # train the model for 20 epochs to update its weights
        model.fit(
            training_dataset,
            epochs=20,
            steps_per_epoch=steps_per_epoch,
            validation_data=validation_dataset,
            validation_steps=validation_steps_per_epoch,
            class_weight=class_weight,
            verbose=2
        )

        # then search for the learning rates
        lrf = LearningRateFinder(model)
        lrf.find(
            training_dataset, 
            startLR=1e-10, 
            endLR=1e-1,
            epochs=30,
            stepsPerEpoch=steps_per_epoch, 
            batchSize=BATCH_SIZE,
            verbose=1
        )

In [ ]:
if FIND_LR:
    lrf.plot_loss()
    !rm -rf /content/lr.hdf5
    sys.exit(0)

# Fine Tuning

In [ ]:
mode='triangular'
step_size=3.
clr_callback = get_lr_callback(mode, LR_MIN, LR_MAX, step_size)
plot_clr(mode, LR_MIN, LR_MAX, step_size, EPOCHS)

# callbacks = get_model_callbacks(VERBOSE_LEVEL, SAVE_OUTPUT, timestamp)
clr_callback = get_lr_callback(mode, LR_MIN, LR_MAX, step_size)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

callbacks = [clr_callback, early_stopping_cb]

In [ ]:
K.clear_session()
with strategy.scope():
    if FIND_LR: # reset the model if we searched for learning rates
        model = reset_model()
    model.trainable = True
    model = compile_model(model)

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    training_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    validation_steps=validation_steps_per_epoch,
    class_weight=class_weight,
    verbose=VERBOSE_LEVEL
)

# Model Evaluation

In [ ]:
example_validation_dataset = get_validation_dataset()
predictions, labels, threshold = evaluate_model(
    model=model, 
    dataset=example_validation_dataset, 
    history=history,
    save_output=SAVE_OUTPUT, 
    timestamp=timestamp
)

# GH & Submission

In [ ]:
%%capture
import os.path
from os import path

if not path.exists("/content/current_metrics.txt"):
    # download the current metrics
    !wget https://raw.githubusercontent.com/SaschaMet/melanoma-classification/master/evaluation/metrics.txt
    !mv /content/metrics.txt.1 /content/current_metrics.txt

In [ ]:
print("current metrics")
with open("/content/current_metrics.txt") as json_file:
    current_metrics = json.load(json_file)
print(current_metrics)

In [ ]:
print("new metrics")
with open("/content/metrics.txt") as json_file:
    metrics = json.load(json_file)
print(metrics)

In [ ]:
# check if the new model is better than the old one
is_better = True

# if one metric is worse than a metric from the prev model, do not update the model
metrics_to_compare = ['f1score', 'precision', 'recall']
for metric in metrics_to_compare:
    if metrics[metric] < current_metrics[metric]:
        is_better = False

print("New model is an improvement compared to the pre model:", is_better)

In [ ]:
if not is_better:
    sys.exit("Stop execution because of inferior model")

In [ ]:
## save the model
model.save_weights("final_weights.hdf5")
model.save("final_model.h5")

In [ ]:
# serialize model to json
json_model = model.to_json()
#save the model architecture to JSON file
with open('model.json', 'w') as json_file:
    json_file.write(json_model)

In [ ]:
# create a new model for the predictions
#K.clear_session()
#with strategy.scope():
    #model = tf.keras.models.load_model("/content/final_model.h5")
    #model.load_weights("final_weights.hdf5")
    #model = compile_model(model)

In [ ]:
test_ds = get_test_dataset(ordered=True)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

In [ ]:
print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds, verbose=1, steps=math.ceil(len(test_ids) / BATCH_SIZE))

In [ ]:
print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

pred_df = pd.DataFrame({'image_name': test_ids, 'target': np.concatenate(probabilities)})
pred_df.head()

In [ ]:
pred_df.to_csv("./submission.csv", index=False)
!kaggle competitions submit -c siim-isic-melanoma-classification -f submission.csv -m "Sub Xception"

In [ ]:
sys.exit("")

In [ ]:
## if the model is an improvement, push it!
!cd /content/melanoma-classification && git checkout -b xception
!cd /content/melanoma-classification && git checkout xception && git reset --hard origin/xception

In [ ]:
!mkdir evaluation

for f in glob.glob("/content/*auc.png"):
    os.rename(f, "auc.png")

for f in glob.glob("/content/*cm.png"):
    os.rename(f, "cm.png")

for f in glob.glob("/content/*history.png"):
    os.rename(f, "history.png")
    
shutil.move("/content/model.json", "/content/evaluation/model.json")
shutil.move("/content/final_model.h5", "/content/evaluation/model.hdf5")
shutil.move("/content/history.png", "/content/evaluation/history.png")
shutil.move("/content/cm.png", "/content/evaluation/cm.png")
shutil.move("/content/auc.png", "/content/evaluation/auc.png")
shutil.move("/content/metrics.txt", "/content/evaluation/metrics.txt")
shutil.move("/content/current_metrics.txt", "/content/evaluation/current_metrics.txt")

!rm -rf /content/melanoma-classification/evaluation
!mv /content/evaluation /content/melanoma-classification/evaluation

In [ ]:
gh_auth = "/content/drive/MyDrive/Colab Notebooks/_auth/gh_auth.txt"
with open(gh_auth) as f:
    gh_auth = f.read()

!cd /content/melanoma-classification && git config --global user.email "sascha.metzger@outlook.com"
!cd /content/melanoma-classification && git config --global user.name "SaschaMet"

!cd /content/melanoma-classification && git remote remove origin
!cd /content/melanoma-classification && git remote add origin $gh_auth

In [ ]:
!cd /content/melanoma-classification && git fetch && git checkout xception && git status

In [ ]:
!cd /content/melanoma-classification && git checkout xception
!cd /content/melanoma-classification && git add .
!cd /content/melanoma-classification && git commit -m "xception"
!cd /content/melanoma-classification && git push -f

In [ ]:
!cd /content/melanoma-classification && git status